# Training Part
Here we train the model

In [ ]:
# Step 1: Install required libraries
!pip install transformers datasets torch sentencepiece

# Step 2: Import necessary libraries
import torch
from datasets import load_dataset
from transformers import MBartForConditionalGeneration, MBartTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments

# Step 3: Load the Dataset
# Load the dataset from Hugging Face
dataset = load_dataset("SKNahin/bengali-transliteration-data")

# Check the available splits
print(dataset)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/300 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/333k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5006 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['bn', 'rm'],
        num_rows: 5006
    })
})


KeyError: 'test'

In [ ]:

# Split the dataset into train and validation subsets (80/20 split)
train_dataset = dataset['train']
val_dataset = train_dataset.train_test_split(test_size=0.2)  # 80/20 split
# Step 4: Data Preprocessing

# Tokenize the data
tokenizer = MBartTokenizer.from_pretrained("facebook/mbart-large-50")

# Define a function for tokenizing both Banglish (source) and Bangla (target) text
def tokenize_function(examples):
    # Tokenize the source text and target text (Banglish -> Bangla)
    source = examples['banglish']
    target = examples['bangla']

    # Tokenize the Banglish text (source)
    inputs = tokenizer(source, padding="max_length", truncation=True, max_length=128, return_tensors="pt")

    # Tokenize the Bangla text (target)
    targets = tokenizer(target, padding="max_length", truncation=True, max_length=128, return_tensors="pt")

    inputs["labels"] = targets["input_ids"]
    return inputs


tokenizer_config.json:   0%|          | 0.00/531 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'MBart50Tokenizer'. 
The class this function is called from is 'MBartTokenizer'.


In [ ]:
# Tokenize the data
tokenizer = MBartTokenizer.from_pretrained("facebook/mbart-large-50")

# Define a function for tokenizing both Banglish (source) and Bangla (target) text
def tokenize_function(examples):
    # Tokenize the source text and target text (Banglish -> Bangla)
    source = examples['rm']  # 'rm' is the Banglish column
    target = examples['bn']  # 'bn' is the Bangla column

    # Tokenize the Banglish text (source)
    inputs = tokenizer(source, padding="max_length", truncation=True, max_length=128, return_tensors="pt")

    # Tokenize the Bangla text (target)
    targets = tokenizer(target, padding="max_length", truncation=True, max_length=128, return_tensors="pt")

    inputs["labels"] = targets["input_ids"]
    return inputs

# Apply tokenization to train and validation datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset['test'].map(tokenize_function, batched=True)

# Step 6: Select a Model
# Select pre-trained mBART model from Hugging Face
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50")


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'MBart50Tokenizer'. 
The class this function is called from is 'MBartTokenizer'.


Map:   0%|          | 0/5006 [00:00<?, ? examples/s]

Map:   0%|          | 0/1002 [00:00<?, ? examples/s]

pytorch_model.bin:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

In [ ]:
# Step 7: Training Setup
# Set up training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",          # Output directory for model checkpoints
    evaluation_strategy="epoch",     # Evaluation strategy
    save_strategy="epoch",          # Save strategy should match evaluation strategy
    learning_rate=2e-5,              # Learning rate
    per_device_train_batch_size=8,   # Batch size for training
    per_device_eval_batch_size=8,    # Batch size for evaluation
    num_train_epochs=3,              # Number of epochs to train
    weight_decay=0.01,               # Weight decay
    logging_dir="./logs",            # Directory for logs
    logging_steps=200,               # Log every 200 steps
    save_steps=200,                  # Save checkpoint every 200 steps (optional)
    load_best_model_at_end=True      # Load the best model at the end of training
)

# Step 8: Training
trainer = Seq2SeqTrainer(
    model=model,                          # The model to train
    args=training_args,                   # Training arguments
    train_dataset=train_dataset,          # Training dataset
    eval_dataset=val_dataset,             # Validation dataset
    tokenizer=tokenizer,                  # Tokenizer
)

# Train the model
trainer.train()

# Step 9: Evaluate the model
# You can evaluate the model on the validation set after training
trainer.evaluate()

# Step 10: Model Inference (Sample Test)
# Test the model by providing a Banglish sentence
def translate_banglish_to_bangla(banglish_text):
    inputs = tokenizer(banglish_text, return_tensors="pt", padding=True, truncation=True)
    output_ids = model.generate(inputs["input_ids"], max_length=128, num_beams=5, early_stopping=True)
    bangla_translation = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return bangla_translation

# Test with a sample Banglish sentence
sample_banglish = "ami khub bhalo achi"
print(f"Banglish: {sample_banglish}")
print(f"Translated Bangla: {translate_banglish_to_bangla(sample_banglish)}")


<ipython-input-6-34c4e2fe4567>:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,0.173100,0.096159
2,0.085500,0.041797
3,0.050500,0.029298


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


Banglish: ami khub bhalo achi


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

# Inference

In [ ]:
import torch
from transformers import MBartForConditionalGeneration, MBartTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments

# Step 1: Load the Model and Tokenizer from Checkpoint
checkpoint_path = "results/checkpoint-1878"

# Load the model (using the config and safetensors model)
model = MBartForConditionalGeneration.from_pretrained(checkpoint_path)

# Load the tokenizer (this should match the one used during training)
tokenizer = MBartTokenizer.from_pretrained(checkpoint_path)

# Step 2: Load Training Arguments and Configuration (if needed for further training)
training_args = Seq2SeqTrainingArguments(
    output_dir=checkpoint_path,  # Set to the checkpoint directory
    overwrite_output_dir=True,  # Overwrite the output directory if resuming training
    do_train=False,  # Set to False if you don't want to continue training
    do_eval=False,   # Set to False if you don't want to evaluate
)

# Step 3: Define the device and move the model to the appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Step 4: Inference or further training

# Sample Banglish text for inference
def translate_banglish_to_bangla(banglish_text):
    inputs = tokenizer(banglish_text, return_tensors="pt", padding=True, truncation=True).to(device)
    output_ids = model.generate(inputs["input_ids"], max_length=128, num_beams=5, early_stopping=True)
    bangla_translation = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return bangla_translation

# Test the translation
sample_banglish = "ami khub bhalo achi"
print(f"Banglish: {sample_banglish}")
print(f"Translated Bangla: {translate_banglish_to_bangla(sample_banglish)}")

Banglish: ami khub bhalo achi
Translated Bangla: আমি খুব ভালো আছি


# Upload to HF

In [ ]:
!pip install huggingface_hub
from huggingface_hub import login
from huggingface_hub import upload_file


login()
# Step 2: Load the model and tokenizer from checkpoint
checkpoint_path = "results/checkpoint-1878"
model = MBartForConditionalGeneration.from_pretrained(checkpoint_path)
tokenizer = MBartTokenizer.from_pretrained(checkpoint_path)

# Step 3: Create a new repository on Hugging Face (if not already created)
repo_name = "banglish-to-bangla"

# Step 4: Push model and tokenizer to Hugging Face Hub
model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)

# Step 5: Optionally, upload additional files like config.json, special_tokens_map.json, etc.
upload_file(
    path_or_fileobj="results/checkpoint-1878/config.json",
    path_in_repo="config.json",
    repo_id=repo_name
)

upload_file(
    path_or_fileobj="results/checkpoint-1878/special_tokens_map.json",
    path_in_repo="special_tokens_map.json",
    repo_id=repo_name
)

upload_file(
    path_or_fileobj="results/checkpoint-1878/sentencepiece.bpe.model",
    path_in_repo="sentencepiece.bpe.model",
    repo_id=repo_name
)
